# SIDCodes → JSON export (using existing utils)

This notebook:
1. Loads the `Mailing.dbo.SIDCodes` table using **`pseudo_anon_utils.load_sid_codes`**.
2. Drops excluded fields (`StudyID`, `PersonID`, `Random`, `OCACID`).
3. Loads the JSON schema using **`common_utils.load_schema`**.
4. Keeps only fields present in the schema and converts rows → JSON records.
5. Validates using **`common_utils.validate_data`**.
6. Exports JSON (either via `common_utils.save_output` or to a local path).


In [1]:
# Imports
import sys
import os
import json
from pathlib import Path
import pandas as pd


sys.path.append(os.path.abspath("N:\\CancerEpidem\\BrBreakthrough\\DeliveryProcess\\Schema_and_Derivation_utils\\Questionnaire\\R0\\scripts"))
from common_utils import get_config, createLogger, load_schema, validate_data, save_output
from pseudo_anon_utils import load_sid_codes

sys.path.append(os.path.abspath("N:\\CancerEpidem\\BrBreakthrough\\DeliveryProcess\\Schema_and_Derivation_utils"))
from config import live_server, Delivery_log_path

In [2]:
# Config
# Assumes common_utils.get_config() is the single source of truth for servers/paths.
config = get_config()

# Choose which SQL server to use
SERVER = config.get(live_server)

logger = createLogger("SIDCodes", Delivery_log_path)

# Schema location
SCHEMA_DIR = r'N:\CancerEpidem\BrBreakthrough\DeliveryProcess\Schema_and_Derivation_utils\Questionnaire\R0\json_schemas\admin_events'
SCHEMA_NAME = 'SIDCodes_Schema'

# Output
LOCAL_OUTPUT_PATH = Path('N:\CancerEpidem\BrBreakthrough\DeliveryProcess\Data_Output_Testing\s5_derived\SIDCodes.json')

# Fields to drop
DROP_FIELDS = ['StudyID', 'PersonID', 'Random', 'Random2', 'OCACID']


In [ ]:
# 1) Load SIDCodes using pseudo_anon_utils
df = load_sid_codes(live_server, logger)
print(f'Loaded {len(df):,} rows and {len(df.columns)} columns')

In [4]:
# 2) Drop excluded fields
df_clean = df.drop(columns=DROP_FIELDS, errors='ignore')
print('Columns after drop:')
print(df_clean.columns.tolist())

Columns after drop:
['JCode', 'KCode', 'LCode', 'MCode', 'NCode', 'PCode', 'QCode', 'RCode', 'SCode', 'TCode', 'ICode', 'Ccode']


In [5]:
# 3) Load schema using common_utils
schema = load_schema(SCHEMA_DIR, SCHEMA_NAME)

# Determine allowed fields from schema
schema_fields = list(schema.get('properties', {}).keys())
print('Schema fields:', schema_fields)

Schema fields: ['KCode', 'JCode', 'MCode', 'LCode', 'NCode', 'TCode', 'PCode', 'QCode', 'RCode', 'SCode', 'ICode', 'Ccode']


In [ ]:
# 4) Keep only schema fields & reshape to JSON records
df_schema = df_clean[[c for c in df_clean.columns if c in schema_fields]].copy()

# enforce column order to match schema
df_schema = df_schema.reindex(columns=schema_fields)

records = df_schema.to_dict(orient='records')
print(f'Prepared {len(records):,} JSON records')

In [7]:
# 5) Validate using common_utils.validate_data
# common_utils.validate_data prints progress and a few sample errors.
validate_data(records, schema, schema_path=SCHEMA_DIR + r'\\' + SCHEMA_NAME + '.json')

Validating 116,673 items...
100% - Validation completed in 23.91 seconds
✓ All items are valid


In [8]:
with open(LOCAL_OUTPUT_PATH, 'w') as f:
    json.dump(records, f, indent=2)
print(f'Wrote {LOCAL_OUTPUT_PATH.resolve()}')

Wrote \\epidemiology\EPIDEMIOLOGY\SHARED\CancerEpidem\BrBreakthrough\DeliveryProcess\Data_Output_Testing\s5_derived\SIDCodes.json
